In [1]:
import torch
torch.set_grad_enabled(False)

In [2]:
# !cp -r /home/galqiwi/root/vcs_projects/linear-layer-compression-Andrei/Andrei/gptq/gptq.py ./
# !cp -r /home/galqiwi/root/vcs_projects/linear-layer-compression-Andrei/Andrei/gptq/edenn.py ./

In [3]:
xtx_dict = torch.load('/home/galqiwi/root/vcs_projects/linear-layer-compression-Andrei/Andrei/gptq/xtx_0.pt')

In [4]:
import transformers

model = transformers.AutoModelForCausalLM.from_pretrained(
    '/mnt/LLM/hub/models--meta-llama--Meta-Llama-3.1-8B/snapshots/48d6d0fc4e02fb1269b36940650a1b7233035cbb/',
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [74]:
weight = model.model.layers[0].self_attn.k_proj.weight
xtx = xtx_dict['self_attn.k_proj']
xtx = torch.diag(10 ** torch.linspace(0, 10, len(xtx)))

In [75]:
weight = weight.cuda()
weight = torch.randn_like(weight)
xtx = xtx.cuda()

In [76]:
def fix_scale(value):
    return value / value.norm() * torch.nn.Linear(*value.shape).weight.norm()

xtx = fix_scale(xtx)

In [77]:
from gptq import apply_gptq
from edenn import edenn, pad_to_block, HadLinear

In [78]:
l_xtx = torch.linalg.cholesky(xtx)

In [79]:
had_block_size = 1024

dequantized_times = HadLinear(apply_gptq(
    weight @ l_xtx, torch.eye(len(xtx), device='cuda'),
    edenn_d=2, edenn_n=1024,
    had_block_size=had_block_size,
), had_block_size)(torch.eye(*xtx.shape, device='cuda')).T.contiguous()

dequantized = HadLinear(apply_gptq(
    weight, xtx,
    edenn_d=2, edenn_n=1024,
    had_block_size=had_block_size,
), had_block_size)(torch.eye(*xtx.shape, device='cuda')).T.contiguous()

GPTQ blocks...:   0%|          | 0/32 [00:00<?, ?it/s]

GPTQ blocks...:   0%|          | 0/32 [00:00<?, ?it/s]

In [80]:
old_relative_mse = ((dequantized_times - weight @ l_xtx).norm() / (weight @ l_xtx).norm()) ** 2
old_relative_mse

tensor(0.0020, device='cuda:0')

In [81]:
old_relative_mse = (
    torch.diag((dequantized_times - weight @ l_xtx) @ (dequantized_times - weight @ l_xtx).T).sum() /
    torch.diag(weight @ xtx @ weight.T).sum()
)
old_relative_mse

tensor(0.0020, device='cuda:0')

In [82]:
mse = (
    (dequantized @ l_xtx - weight @ l_xtx).norm() /
    weight.norm()
) ** 2 / (torch.logdet(xtx) / len(xtx)).exp().item()

mse

tensor(3.9711, device='cuda:0')

In [90]:
mse = (
    (dequantized @ l_xtx - weight @ l_xtx).norm() /
    weight.norm()
) ** 2 / torch.linalg.svd(xtx).S.max()

mse

tensor(3.9711e-05, device='cuda:0')

In [83]:
mse = (
    (dequantized @ l_xtx - weight @ l_xtx).norm() /
    (weight @ l_xtx).norm()
) ** 2
mse

tensor(0.0009, device='cuda:0')

In [ ]:
mse = (
    (dequantized @ l_xtx - weight @ l_xtx).norm() /
    weight.norm()
) ** 2 / (torch.logdet(xtx) / len(xtx)).exp().item()

mse

In [95]:
(dequantized @ l_xtx - weight @ l_xtx).norm() / (weight).norm()

tensor(0.0125, device='cuda:0')

In [98]:
(eigenvalues < 0.0125).float().mean()

tensor(0.7505, device='cuda:0')

In [91]:
eigenvalues = torch.linalg.svd(xtx).S

In [92]:
eigenvalues

tensor([3.9053e+00, 3.8834e+00, 3.8616e+00,  ..., 3.9494e-10, 3.9273e-10,
        3.9053e-10], device='cuda:0')